In [1]:
import pandas as pd
import sys
sys.path.insert(0,r"C:\Users\LENOVO\Downloads")
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.set_option('display.max_row', None)
pd.set_option('display.max_column', None)

In [3]:
base_noroccidente = pd.read_csv(r'C:\Users\LENOVO\Desktop\base_noroccidente.txt', sep = ",", engine = "python")

In [4]:
base2_noroccidente = base_noroccidente[['SUSCNITC','SERVDESC','SESUCUSA','SESUCICL','SESULOCA',
                                  'TIPIFICACION_CLIENTE','CALIFICACION_VECTOR_SERVICIO',
                                  'CANAL_ENVIO','SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR',
                                  'ANTIGUEDAD_DIAS','CANAL_PAGO','SESUSAPE','SESUSAAN','PROM_SUSC']]
base3_noroccidente = base2_noroccidente.dropna()

In [5]:
tmp = base3_noroccidente[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
tmp.columns

group_nit = tmp.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()
group_nit = group_nit.rename(columns={"CALIFICACION_VECTOR_SERVICIO":"PROM_CALIFICACION"})

In [6]:
base4_noroccidente = pd.merge(base3_noroccidente, group_nit, how='left', on=['SUSCNITC'])

In [7]:
base4_noroccidente['y'] = pd.cut(x=base4_noroccidente['PROM_CALIFICACION'],
                            bins=[-1,50,76,100],
                            labels=['No Pago','Pago Inoportuno','Pago'])

In [8]:
base5_noroccidente = base4_noroccidente[['SERVDESC','SESUCUSA','SESUCICL','SESULOCA','TIPIFICACION_CLIENTE',
                       'CALIFICACION_VECTOR_SERVICIO','CANAL_ENVIO','SEGMENTACION','ESTRATO_AGRUPADO',
                       'DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO','SESUSAPE','SESUSAAN','PROM_SUSC','y']]

In [9]:
names = base5_noroccidente.columns

for x in names:
    globals()['FREQ_%s' % x]= base5_noroccidente[x].value_counts().rename_axis([x]).reset_index(name='F_{x}'.format(x=x))

In [10]:
freq_noroccidente = base5_noroccidente.merge(FREQ_SERVDESC,on="SERVDESC").merge(FREQ_SESUCUSA,on='SESUCUSA').merge(FREQ_SESUCICL,on='SESUCICL').merge(FREQ_SESULOCA,on='SESULOCA').merge(FREQ_TIPIFICACION_CLIENTE,on='TIPIFICACION_CLIENTE').merge(FREQ_CANAL_ENVIO,on='CANAL_ENVIO').merge(FREQ_SEGMENTACION,on='SEGMENTACION').merge(FREQ_ESTRATO_AGRUPADO,on='ESTRATO_AGRUPADO').merge(FREQ_DEPARTAMENTO_AGR,on='DEPARTAMENTO_AGR').merge(FREQ_CANAL_PAGO,on='CANAL_PAGO')

In [13]:
freq_noroccidente2 = freq_noroccidente[['F_SERVDESC', 'F_SESUCUSA',
       'F_SESUCICL', 'F_SESULOCA', 'F_TIPIFICACION_CLIENTE', 'F_CANAL_ENVIO',
       'F_SEGMENTACION', 'F_ESTRATO_AGRUPADO', 'F_DEPARTAMENTO_AGR',
       'F_CANAL_PAGO']]

In [14]:
freq_noroccidente2['AVG_FREQ'] = freq_noroccidente2.mean(axis=1)

In [15]:
n, m = freq_noroccidente2.shape
n_reg_del = round((0.05 * n) /100)
outliers = freq_noroccidente2.nsmallest(n_reg_del, 'AVG_FREQ')
outliers_idx = outliers.index
outliers_idx = outliers_idx.sort_values(ascending=False)

In [16]:
for x in outliers_idx:
    base5_noroccidente = base5_noroccidente.drop([x])

In [17]:
dataTypeSeries = base5_noroccidente.dtypes
dataTypeSeries

cat = base5_noroccidente.select_dtypes(include=['object','category'])
cuanti = base5_noroccidente.select_dtypes(include=['int64','float64'])
  
cat = cat.apply(lambda col: LabelEncoder().fit_transform(col))
base5_noroccidente = cuanti.join(cat)

In [18]:
X = base5_noroccidente[['SERVDESC','SESUCUSA','SESUCICL','SESULOCA','TIPIFICACION_CLIENTE','CANAL_ENVIO',
               'SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO',
               'SESUSAPE','SESUSAAN','PROM_SUSC']]

y = base5_noroccidente['y']

In [19]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
#X_ = X
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

In [20]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
#X_fin = X[['SESUSERV','SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 'PROM_SUSC',
#'CANAL_ENVIO','SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]
X_fin = X
X_ = StandardScaler().fit_transform(X_fin)

for train_index, test_index in kf.split(X_, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Entrenamiento de la red MLP
    model = KNeighborsClassifier(7).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                      y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

[             precision    recall  f1-score   support      pred       AUC
0             0.692817  0.579257  0.630968   36331.0   30376.0  0.882819
1             0.872602  0.935256  0.902843  406356.0  435533.0  0.861589
2             0.652306  0.525002  0.581771  118990.0   95768.0  0.813139
avg / total   0.814304  0.825318  0.817239  561677.0  561677.0  0.925922,              precision    recall  f1-score   support      pred       AUC
0             0.691783  0.566553  0.622936   36332.0   29755.0  0.873513
1             0.873742  0.942786  0.906952  406355.0  438466.0  0.872459
2             0.670808  0.526859  0.590183  118990.0   93456.0  0.826359
avg / total   0.818981  0.830337  0.821474  561677.0  561677.0  0.929658,              precision    recall  f1-score   support      pred       AUC
0             0.696190  0.588462  0.637809   36332.0   30710.0  0.882746
1             0.875291  0.933721  0.903562  406355.0  433481.0  0.863344
2             0.649396  0.532036  0.584887  1189

In [21]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

array([[ 20664,   5825,   9843],
       [  6204, 372583,  27568],
       [  7230,  48892,  62867]], dtype=int64)